In [ ]:
数据介绍：将根据用户的位置，准确性和时间戳预测用户正在查看的业务。

train.csv，test.csv 
row_id：登记事件的ID
xy：坐标
准确性：定位准确性 
时间：时间戳
place_id：业务的ID，这是您预测的目标

In [ ]:
步骤分析
对于数据做一些基本处理（这里所做的一些处理不一定达到很好的效果，我们只是简单尝试，有些特征我们可以根据一些特征选择的方式去做处理）

    1 缩小数据集范围 DataFrame.query()

    2 选取有用的时间特征

    3 将签到位置少于n个用户的删除

分割数据集

标准化处理

k-近邻预测

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
# 1.获取数据  pd.read_csv
data = pd.read_csv('./data/FBlocation/train.csv')

In [5]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [6]:
data.shape

(29118021, 6)

In [7]:
# 1.获取数据  pd.read_csv

# 2.数据基本处理  提取时间特征，剔除入住人数少的酒店，选择一部分数据进行建模演示， 划分数据集

# 3.特征工程 标准化

# 4.机器学习 交叉验证和网格搜索

# 5.模型评估 准确率

In [8]:
# 2.数据基本处理  提取时间特征，剔除入住人数少的酒店，选择一部分数据进行建模演示， 划分数据集
# 2.1选择一部分数据进行建模演示 x>2&x<2.5&y<2&y>2.5
data = data.query('x>2 & x<2.5 & y>2 & y<2.5')

In [9]:
data.head()

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909


In [10]:
data.shape

(71664, 6)

In [11]:
# 2.2 提取时间特征
date = pd.to_datetime(data['time'], unit='s')
date = pd.DatetimeIndex(date)
data['day'] = date.day
data['weekday'] = date.weekday
data['hour'] = date.hour

In [12]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
163,163,2.1663,2.3755,84,669737,3869813743,8,3,18
310,310,2.3695,2.2034,3,234719,2636621520,3,5,17
658,658,2.3236,2.1768,66,502343,7877745055,6,1,19
1368,1368,2.2613,2.3392,73,319822,9775192577,4,6,16
1627,1627,2.3331,2.0011,66,595084,6731326909,7,2,21


In [13]:
data.shape

(71664, 9)

In [14]:
# 2.3剔除入住人数少的酒店
place_count = data.groupby(by='place_id').count()

In [16]:
place_count.head()

,row_id,x,y,accuracy,time,day,weekday,hour
place_id,,,,,,,,
1006234733,1,1,1,1,1,1,1,1
1008823061,4,4,4,4,4,4,4,4
1012580558,3,3,3,3,3,3,3,3
1025585791,21,21,21,21,21,21,21,21
1026507711,220,220,220,220,220,220,220,220


In [17]:
place_count = data.groupby(by='place_id').count()['row_id']

In [18]:
place_count.head()

place_id
1006234733      1
1008823061      4
1012580558      3
1025585791     21
1026507711    220
Name: row_id, dtype: int64

In [19]:
place_count = place_count[place_count>3]

In [21]:
place_count.head()

place_id
1008823061      4
1025585791     21
1026507711    220
1032417180     10
1040557418    123
Name: row_id, dtype: int64

In [28]:
data['place_id'].isin(place_count.index).head()

163     True
310     True
658     True
1368    True
1627    True
Name: place_id, dtype: bool

In [24]:
data = data[data['place_id'].isin(place_count.index)]

In [25]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
163,163,2.1663,2.3755,84,669737,3869813743,8,3,18
310,310,2.3695,2.2034,3,234719,2636621520,3,5,17
658,658,2.3236,2.1768,66,502343,7877745055,6,1,19
1368,1368,2.2613,2.3392,73,319822,9775192577,4,6,16
1627,1627,2.3331,2.0011,66,595084,6731326909,7,2,21


In [29]:
# 2.4 划分数据集
# 提取特征值和目标值
x = data[['x', 'y', 'accuracy', 'day', 'weekday', 'hour']]
y = data['place_id']
# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)   # 参数：特征值， 目标值，测试集占的比例

In [30]:
# 3.特征工程 标准化
# 3.1 初始化转换器
transfer = StandardScaler()

In [31]:
# 3.2 转换数据
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [32]:
x_train

array([[-1.30547088,  0.8580328 ,  0.69169019,  0.62405331, -0.61485018,
         0.06343705],
       [ 1.35751353, -0.26600542,  0.71755385, -0.5075462 ,  1.69463078,
         1.66081834],
       [ 1.50305723,  0.95669675, -0.3859623 , -0.13034637, -1.76959065,
        -0.95307832],
       ...,
       [ 0.77866167,  0.95596591, -0.22215912,  1.37845298,  0.5398903 ,
        -0.80786184],
       [-1.7600457 , -0.95738394, -0.66184133,  1.00125314, -0.03747994,
         1.37038538],
       [ 0.04363278, -0.15710965, -0.15318936, -0.5075462 ,  1.69463078,
         0.78951945]])

In [39]:
# 4.机器学习 交叉验证和网格搜索
# 4.1 初始化估计器  
estimator = KNeighborsClassifier(algorithm='ball_tree')

In [45]:
# 4.2 交叉验证和网格搜索
# 4.2.1 构建参数字典
param_dict = {'n_neighbors':[1,3,5,7,9]}
# 4.2.2 初始化GSCV对象
estimator_gscv = GridSearchCV(estimator=estimator, param_grid=param_dict, cv=6)

In [46]:
# 训练模型
estimator_gscv.fit(x_train, y_train) # 参数：训练集的特征值，训练集的目标值

/home/python/.virtualenvs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=6.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=6, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [47]:
# 5.模型评估 准确率
estimator_gscv.score(x_test, y_test)  # 参数：测试集的特征值， 测试集的目标值

0.3700281527466975

In [48]:
# 获取最优模型
estimator_gscv.best_estimator_

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [49]:
# 获取最优超参数
estimator_gscv.best_params_

{'n_neighbors': 5}